#### CIE4604 Simulation and Visualization

# Module 2 Satellite Orbits - Exercise 4

**Hans van der Marel, 21 November 2020**

The function `tle2vec` in the Python module `tleplot.py` returns the position and velocity of satellites in an ECI reference frame. However, often it is more convenient to have the position and velocity in an ECEF reference frame. The transformation between an ECI and ECEF, and vice versa, is done with the functions `eci2ecef` and `ecef2eci` from the `crsutil.py` module.

In this example we explain the theory behind them, and have a look at the effect on the look angles.

## Import crsutil.py and tleplot.py Python modules

For the first time, download `CIE4604-M2-python.zip` from Brightspace and unzip this in your current working directory. This should give you two Python modules: `crsutil.py` and `tleplot.py`.

For this Jupyter notebook to work, the two Python modules should be in the same folder as this notebook. Import the modules using the following statements:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import crsutil as crs
import tleplot as tle
from scipy.interpolate import interp1d 

To get the docs for a specific function, use `help(functionname)`, or, to get the docs on all the functions at once do `help(module)`. To get help in a separate window type `module?` or `functionname?`. 


## Read TLE for Earth Resource Satellites

First read the Two Line Elements (TLE) for Earth Resource Satellite, that contain Radarsat-2 (amongst others), and save the 
results in a structure array *tleERS* with `tleread`

In [ ]:
tleERS = tle.tleread('resource-10-oct-2017.tle', verbose=0)

To suppress output during the reading we have added the option `verbose=0`
to the call of `tleread`.

## Compute satellite position and velocity for Radarsat-2 in ECI

Compute the positions and velocities for Radarsat-2 in the ECI reference frame for a 24 hour period with samples every minute.

In [ ]:
t = tle.tledatenum(['2017-10-10 0:00', 24*60, 1])
xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')

## Conversion from ECI to ECEF and vice-versa

To convert the positions from ECI to ECEF we use the fuction `eci2ecef` from the `crsutil.py` module 

In [ ]:
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

To convert from ECEF to ECI use `ecef2eci`. We test this out by transforming the previously computed coordinates in ECEF back to ECI, and compare with the original values

In [ ]:
xsat2, vsat2 = crs.ecef2eci(t, xsate, vsate)

print('max diff xsat: ', np.max(np.max(np.abs(xsat-xsat2))))
print('max diff vsat: ', np.max(np.max(np.abs(vsat-vsat2))))

## Conversion from ECI to ECEF - explained

The conversion from ECI to ECEF consists of a number of steps. The 
first step is to compute the Earth Rotation Angle(GMST) with respect
to the vernal equinox.

To save compute resources we compute first GMST at a reference epoch *t[0]*, 
and then use the Earth rotation rate to compute GMST at the others epochs 


In [ ]:
gst0, omegae = crs.ut2gmst(t[0])
gst = gst0 + 2 * np.pi * omegae * (t-t[0])

The next step is to rotate satellite positions around the z-axis (ECI->ECEF))

In [ ]:
xsate = np.zeros(xsat.shape)
xsate[:, 0] = np.cos(gst)*xsat[:, 0] + np.sin(gst) * xsat[:, 1]
xsate[:, 1] = -np.sin(gst) * xsat[:, 0] + np.cos(gst) * xsat[:, 1]
xsate[:, 2] = xsat[:, 2]

To convert velocity is more complicated. The velocity in ECEF
consists of two parts. We find this by differentiating the transformation
formula for the positions
```
    xsate = R * xsat
```
This gives (product rule, and some rewriting), with *_dot* the derivatives
```
    xsate_dot = R * xsat_dot + R_dot * xsat    <=>
    vsate = R * ( vsat + inv(R)*R_dot * xsat ) <=>
    vsate = R * ( vsat + W * xsat )
```
with 
```
    W = inv(R) * R_dot = [[ 0 -w[2] w[1] ], 
                          [ w[2] 0 -w[0] ],
                          [-w[1] w[0] 0 \]]
```
and with *w\[2\]* the angular velocity vector of the ECEF frame with respect to
the ECU frame, expressed in the ECI frame.

The velocity vector in the ECEF is computed as follows...

In [ ]:
w = np.array([[0],
              [0],
              [2*np.pi*omegae/86400]])

W = np.array([[0, -w[2, 0], w[1, 0]],
              [w[2, 0], 0, -w[0, 0]],
              [-w[1, 0], w[0, 0], 0]])

vsatw = vsat - np.matmul(xsat, W.T)

vsate = np.zeros(vsat.shape)
vsate[:, 0] = np.cos(gst)*vsatw[:, 0] + np.sin(gst) * vsatw[:, 1]
vsate[:, 1] = -np.sin(gst) * vsatw[:, 0] + np.cos(gst) * vsatw[:, 1]
vsate[:, 2] = vsatw[:, 2]

If we compare the velocities with the numerical derivative of positions
we are in good agreement for both the ECI and ECEF frames.

In [ ]:
# Prepare time arrays for differencing
tnum = ( t[0:-1] + t[1:] ) / 2                           # average of two epochs
tdiff = np.diff(t) * 24 * 3600                           # first differences of time, converted to seconds 

# Numerical derivative of the ECI and ECEF positions
vsatnum = np.diff(xsat, axis=0)
vsatnum = vsatnum / tdiff[:, np.newaxis]
vsatenum = np.diff(xsate, axis=0)
vsatenum = vsatenum / tdiff[:, np.newaxis]

# Plot velocity in ECI
plt.figure()
plt.plot(t, vsat)
plt.title('Velocities in ECI')
plt.ylabel('Velocity [m/s/]')

# Plot difference of the numerical velocities with actual velocities in ECI
plt.figure()
for k in range(3):
    set_interp = interp1d(t, vsat[:,k], kind='linear')
    vsatdiff = vsatnum[:,k] - np.array([set_interp(tnum)])
    plt.plot(tnum, vsatdiff.T)

plt.title('Numerical velocity difference in ECI')
plt.ylabel('Difference [m/s/]')

# Plot difference of the numerical velocities with actual velocities in ECEF
plt.figure()
for k in range(3):
    set_interp = interp1d(t, vsate[:,k], kind='linear')
    vsatdiff = vsatenum[:,k] - np.array([set_interp(tnum)])
    plt.plot(tnum, vsatdiff.T)

plt.title('Numerical velocity difference in ECEF (rotation term included)')
plt.ylabel('Difference [m/s/]')
plt.show()

When the term for frame rotation is not included the velocities will not be correct, as is demonstrated by the following code


In [ ]:
vsate_nr = np.zeros(vsat.shape)
vsate_nr[:, 0] = np.cos(gst) * vsat[:, 0] + np.sin(gst) * vsat[:, 1]
vsate_nr[:, 1] = -np.sin(gst) * vsat[:, 0] + np.cos(gst) * vsat[:, 1]
vsate_nr[:, 2] = vsat[:, 2]

# Plot difference of the numerical velocities with actual velocities in ECEF (w/o rotations)
plt.figure()
for k in range(3):
    set_interp = interp1d(t, vsate_nr[:,k], kind='linear')
    vsatdiff = vsatenum[:,k] - np.array([set_interp(tnum)])
    plt.plot(tnum, vsatdiff.T)

plt.title('Numerical velocity difference in ECEF (w/o rotation term)')
plt.ylabel('Difference [m/s/]')
plt.show()

This is one of the reason that most computations with satellite positions
and velocities are done in the ECI reference frame, even if this means a 
rotating observer position (in ECI). 

## Redo test case of exercise 3 

To study the effect on the lookangles we redo the test case with Radarsat-2 of exercise 3. We select a couple of epochs for which we know
Radarsat-2 is in the proximity of Delft.

First we compute the ECEF coordinates for Delft.

In [ ]:
# Specify station position (latitude, longitude, height) 

objcrd=[ 52, 4.8,  0 ]

# Convert input to spherical coordinates in radians/meters

Re = 6378136                     # [m]   radius of the Earth 

lat = objcrd[0] * np.pi/180      # convert latitude from degrees to radians
lon = objcrd[1] * np.pi/180      # convert longitude from degrees to radians
Rs = Re + objcrd[2]              # convert height to radius (from CoM) using mean Earth radius Re 

# Position of the observer in ECEF (assume latitude and longitude are for spherical Earth)

xobje = Rs * np.array([ np.cos(lat) * np.cos(lon),  np.cos(lat) * np.sin(lon),  np.sin(lat)]) 
vobje = np.array([ 0, 0, 0])

print('xobje',xobje)

#### Lookangles in ECEF

Next we compute the satellite positions in ECI, convert them to ECEF using the function `eci2ecef` and print the lookangles.

To compute the lookangles we use the function `satlookanglesp` from the `crsutil.py` module that were introduced in example 3. The function `prtlookangle` is used to print a nice table.

In [ ]:
t = tle.tledatenum(['2017-9-28 6:00',10,1])

xsat, vsat = tle.tle2vec(tleERS, t, 'RADARSAT-2')
xsate, vsate = crs.eci2ecef(t, xsat, vsat)

lookangles, flags = crs.satlookanglesp(t, np.hstack([xsate, vsate]), xobje)
crs.prtlookangle(t, lookangles, flags)

#### Look angles in ECI

For the object coordinates *xobje* only the position is enough in an ECEF frame, but for an ECI frame both position and velocity need to be given for every epoch.


In [ ]:
xobj, vobj = crs.ecef2eci(t, xobje)

lookangles, flags = crs.satlookanglesp(t, np.hstack([xsat, vsat]), np.hstack([xobj, vobj]))
crs.prtlookangle(t, lookangles, flags)

As you can see for most quantities it doesn't matter with reference frame
is used, except for the satellite heading and look angle with respect to the flight direction! 

The reason for
this is that the frame rotation influences the computation of the
heading; as a matter of fact, we should do an extra correction to account
for the rotation of the frame.

This is also one of the reasons that most computations with satellite positions
and velocities are done in the ECI reference frame, even if this means a 
rotating observer position (in ECI). In general, it is a good idea to do 
this sort of computations in an ECI reference frame, where the laws
of Newton mechanics hold, and convert it into an ECEF reference frame 
only at the last moment (if needed).

[End of this Jupyter notebook]